In [1]:
import pandas as pd
import html
import re
from unidecode import unidecode 

In [2]:
# Load the datasets
books_df = pd.read_csv("Books.csv", encoding='latin1')
ratings_df = pd.read_csv("Ratings.csv", encoding='latin1')

/var/folders/95/csh3_2vs4nb11f7sndt720_w0000gn/T/ipykernel_4278/945850662.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books_df = pd.read_csv("Books.csv", encoding='latin1')


In [3]:
users_df = pd.read_csv("Users.csv", encoding='latin1')

In [4]:
# Display basic info about each dataset
books_df.info()
ratings_df.info()
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), obje

In [5]:
# Preview the first few rows the dataset
books_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [6]:
# Drop the image URL columns from the books dataset
books_df = books_df.drop(columns=['Image-URL-S', 'Image-URL-M', 'Image-URL-L'])

In [7]:
# Verify the result
books_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [8]:
ratings_df = ratings_df.sort_values(by='User-ID', ascending=True)
ratings_df = ratings_df.reset_index(drop=True)
ratings_df.head()

,User-ID,ISBN,Book-Rating
0,2,0195153448,0
1,7,034542252,0
2,8,0771025661,0
3,8,1881320189,7
4,8,1575663937,6


In [9]:
users_df.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [10]:
users_df[['City', 'State', 'Country']] = users_df['Location'].str.split(',', n=2, expand=True)
users_df = users_df.drop(columns='Location')

In [11]:
# Keep only users who appear in the Ratings dataset
valid_user_ids = ratings_df['User-ID'].unique()
users_df = users_df[users_df['User-ID'].isin(valid_user_ids)]

In [12]:
# Remove unrealistic ages
users_df = users_df[(users_df['Age'] >= 5) & (users_df['Age'] <= 100)]

# (Optional) Impute missing ages with median
users_df['Age'] = users_df['Age'].fillna(users_df['Age'].median())
users_df = users_df.dropna(subset=['Age'])

In [13]:
users_df['Age'] = users_df['Age'].astype(int)
users_df = users_df.reset_index(drop=True)
users_df.head()

,User-ID,Age,City,State,Country
0,2,18,stockton,california,usa
1,10,26,albacete,wisconsin,spain
2,19,14,weston,,
3,20,19,langhorne,pennsylvania,usa
4,36,24,montreal,quebec,canada


In [14]:
users_df[['City', 'State', 'Country']] = users_df[['City', 'State', 'Country']].replace(r'^\s*$', pd.NA, regex=True)
users_df = users_df.dropna(subset=['City', 'State', 'Country'])
users_df = users_df.reset_index(drop=True)

In [15]:
users_df[['City', 'State', 'Country']].isna().sum()

City       0
State      0
Country    0
dtype: int64

In [16]:
users_df.head()

,User-ID,Age,City,State,Country
0,2,18,stockton,california,usa
1,10,26,albacete,wisconsin,spain
2,20,19,langhorne,pennsylvania,usa
3,36,24,montreal,quebec,canada
4,38,34,viterbo,lazio,italy


In [17]:
# Convert to numeric, set invalid entries to NaN
books_df['Year-Of-Publication'] = pd.to_numeric(books_df['Year-Of-Publication'], errors='coerce')

# Set anything outside 1700–2025 to NaN
books_df.loc[
    (books_df['Year-Of-Publication'] < 1700) | 
    (books_df['Year-Of-Publication'] > 2025),
    'Year-Of-Publication'
] = pd.NA

books_df = books_df.dropna(subset=['Year-Of-Publication'])
books_df = books_df.reset_index(drop=True)

In [18]:
books_df['Year-Of-Publication'] = books_df['Year-Of-Publication'].astype('Int64')

In [19]:
# Keep only books that have at least one rating
books_df = books_df[books_df['ISBN'].isin(ratings_df['ISBN'])].reset_index(drop=True)

In [20]:
def has_non_ascii(text):
    return bool(re.search(r'[^\x00-\x7F]', str(text)))

# Add helper flags to check for encoding issues
books_df['title_corrupt'] = books_df['Book-Title'].apply(has_non_ascii)
books_df['author_corrupt'] = books_df['Book-Author'].apply(has_non_ascii)
books_df['publisher_corrupt'] = books_df['Publisher'].apply(has_non_ascii)

books_df['short_title'] = books_df['Book-Title'].str.len() <= 2
books_df['short_author'] = books_df['Book-Author'].str.len() <= 2
books_df['short_publisher'] = books_df['Publisher'].str.len() <= 2

In [21]:
books_df_cleaned = books_df[
    ~(
        ((books_df['short_title'] & books_df['title_corrupt']) |
         (books_df['short_author'] & books_df['author_corrupt']) |
         ((books_df['short_publisher'] | books_df['publisher_corrupt']) &
          (books_df['title_corrupt'] | books_df['author_corrupt'])))  # Only drop if publisher is bad AND others too
    )
].reset_index(drop=True)

print("Before:", len(books_df))
print("After:", len(books_df_cleaned))
print("Dropped:", len(books_df) - len(books_df_cleaned))

Before: 265537
After: 264637
Dropped: 900


In [22]:
books_df_cleaned = books_df_cleaned.drop(
    columns=[
        'title_corrupt', 'author_corrupt', 'publisher_corrupt',
        'short_title', 'short_author', 'short_publisher'
    ]
)

In [23]:
books_df_cleaned = books_df_cleaned.dropna(subset=['Book-Title', 'Book-Author', 'Publisher'])
books_df_cleaned = books_df_cleaned.reset_index(drop=True)

In [24]:
books_df_cleaned.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [25]:
books_df_cleaned['Publisher'] = books_df_cleaned['Publisher'].apply(html.unescape)
books_df_cleaned['Book-Title'] = books_df_cleaned['Book-Title'].apply(html.unescape)
books_df_cleaned['Book-Author'] = books_df_cleaned['Book-Author'].apply(html.unescape)

In [26]:
def split_camel_case(text):
    return re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', text)

# Apply to Publisher column
books_df_cleaned['Publisher'] = books_df_cleaned['Publisher'].apply(split_camel_case)
books_df_cleaned['Book-Author'] = books_df_cleaned['Book-Author'].apply(split_camel_case)
books_df_cleaned['Book-Title'] = books_df_cleaned['Book-Title'].apply(split_camel_case)

In [27]:
books_df_cleaned['Book-Title'] = books_df_cleaned['Book-Title'].str.title()
books_df_cleaned['Book-Author'] = books_df_cleaned['Book-Author'].str.title()
books_df_cleaned['Publisher'] = books_df_cleaned['Publisher'].str.title()

In [28]:
CANON = {
    # W. W. Norton
    "w w norton & co": "W. W. Norton & Company",
    "w. w. norton & co": "W. W. Norton & Company",
    "w w norton & company": "W. W. Norton & Company",
    "w w norton & co inc": "W. W. Norton & Company",
    "w. w. norton & company": "W. W. Norton & Company",

    # Little, Brown
    "little brown": "Little, Brown And Company",
    "little brown & co": "Little, Brown And Company",
    "little brown & company": "Little, Brown And Company",
    "little brown and company": "Little, Brown And Company",
    "little, brown": "Little, Brown And Company",
    "little, brown & company": "Little, Brown And Company",

    # Knopf
    "knopf": "Alfred A. Knopf",
    "a. a. knopf": "Alfred A. Knopf",
    "alfred a. knopf": "Alfred A. Knopf",

    # Random House
    "random house inc": "Random House",
    "random house, inc.": "Random House",
    "random house canada": "Random House",
    "random house value publishing": "Random House",
    "random house": "Random House",

    # HarperCollins
    "harpercollins": "HarperCollins",
    "harper collins": "HarperCollins",
    "harper collins - uk": "HarperCollins",
    "harper collins publishers": "HarperCollins",
    "harpercollins publisher": "HarperCollins",
    "harpercollins juvenile books": "HarperCollins",
    "harpercollins library": "HarperCollins",

    # Penguin / Putnam
    "penguin usa": "Penguin Books",
    "penguin usa (paper)": "Penguin Books",
    "penguin books ltd": "Penguin Books",
    "penguin books": "Penguin Books",
    "penguin putnam~trade": "Penguin Putnam",
    "penguin putnam": "Penguin Putnam",

    # Simon & Schuster
    "simon & schuster (trade division)": "Simon & Schuster",
    "simon schuster trade": "Simon & Schuster",
    "simon pulse": "Simon & Schuster",
    "simon & schuster": "Simon & Schuster",

    # St. Martin’s
    "st. martin's paperbacks": "St. Martin's Press",
    "st. martin's griffin": "St. Martin's Press",
    "st. martin's minotaur": "St. Martin's Press",

    # Bantam / Doubleday / Dell / Ballantine
    "bantam books": "Bantam",
    "bantam dell pub group": "Bantam",
    "bantam dell publishing group": "Bantam",
    "bantam doubleday dell": "Bantam",
    "doubleday books": "Doubleday",
    "ballantine books": "Ballantine",

    # Tor / Vintage / Anchor / Putnam
    "tor fantasy": "Tor Books",
    "tor books (mm)": "Tor Books",
    "tor books": "Tor Books",
    "vintage books usa": "Vintage",
    "vintage": "Vintage",
    "anchor pub": "Anchor",
    "anchor books": "Anchor",
    "putnam pub group": "Putnam",
    "putnam pub group (paper)": "Putnam",
    "putnam publishing group": "Putnam",
    "g. p. putnam's sons": "Putnam",

    # Scholastic / Avon / Pocket
    "scholastic inc": "Scholastic",
    "scholastic": "Scholastic",
    "avon books": "Avon",
    "pocket books": "Pocket Books",

    # McGraw-Hill etc.
    "mc graw-hill": "McGraw-Hill",
    "mcgraw-hill": "McGraw-Hill",

    # German TB variants
    "rowohlt tb.": "Rowohlt",
    "rowohlt, reinbek": "Rowohlt",
    "ullstein tb": "Ullstein",
    "fischer (tb.), frankfurt": "Fischer",
}

# labels that are clearly not useful as publishers
DROP_SET = {
    "null","none","n/a","na","unknown","---------","—","-","*", "too far", "image"
}

# chunks in parentheses/brackets to drop (format/region/city noise)
PAREN_DROP_HINTS = re.compile(
    r'\b(mm|pb|paper(back)?|hardcover|mass market|uk|usa|canada|aus|ny|nyc|london|frankfurt|paris|tokyo|paper|large print)\b',
    re.IGNORECASE
)

ACRONYM_KEEP = {
    "USA","US","UK","UAE","LLC","INC","LTD","GP","TB","MM"
}

def _normalize_spaces(s: str) -> str:
    return re.sub(r'\s{2,}', ' ', s).strip()

def _strip_mojibake(s: str) -> str:
    return None if re.search(r'[ÃÂð¼½¾þÿÞß]', s or '') else s

def _strip_paren_noise(s: str) -> str:
    # remove ()/[] chunks that look like format or location tags
    def repl(m):
        chunk = m.group(1)
        return '' if PAREN_DROP_HINTS.search(chunk) else m.group(0)
    prev = None
    while s != prev:
        prev = s
        s = re.sub(r'\(([^()]*)\)', repl, s)
        s = re.sub(r'\[([^\[\]]*)\]', repl, s)
    return _normalize_spaces(s)

def _std_ampersand_and_commas(s: str) -> str:
    s = re.sub(r'\s*&\s*', ' & ', s)
    s = re.sub(r'\s*,\s*', ', ', s)
    s = s.replace(' ,', ',')
    return _normalize_spaces(s)

def _titlecase_preserving_acronyms(s: str) -> str:
    out = []
    for token in s.split():
        raw = token.strip(",.")
        if raw.upper() in ACRONYM_KEEP:
            out.append(token.replace(raw, raw.upper()))
        else:
            # title-case each hyphen part
            parts = re.split(r'(-)', token)
            parts = [p if p == '-' else (p[:1].upper() + p[1:].lower()) for p in parts]
            out.append(''.join(parts))
    return ' '.join(out)

def _final_tidy(s: str) -> str:
    s = re.sub(r'\s*\.\s*', '. ', s)
    s = _normalize_spaces(s).strip(" ,.")
    return s

def clean_publisher(x):
    """Normalize publisher names, drop junk, map to canonical labels."""
    if not isinstance(x, str) or not x.strip():
        return None
    s = x.strip()

    # kill mojibakey rows
    s = _strip_mojibake(s)
    if s is None:
        return None

    # normalize encoding / spacing
    s = unidecode(s)
    s = _normalize_spaces(s)
    s = _std_ampersand_and_commas(s)

    # unify 'and' to '&' when standalone word
    s = re.sub(r'\bAnd\b', '&', s, flags=re.IGNORECASE)
    s = _std_ampersand_and_commas(s)

    # remove obvious format/location in parentheses/brackets
    s = _strip_paren_noise(s)

    # drop total placeholders or nearly-empty tokens
    if s.lower() in DROP_SET or re.fullmatch(r'[-*_.]+', s):
        return None

    # light normalization of common company tokens
    s = re.sub(r'\bCo\b\.?', 'Co.', s, flags=re.IGNORECASE)
    s = re.sub(r'\bInc\b\.?', 'Inc.', s, flags=re.IGNORECASE)
    s = re.sub(r'\bLtd\b\.?', 'Ltd.', s, flags=re.IGNORECASE)
    s = re.sub(r'\bPub(?:lishing)?\s+Group\b', 'Publishing Group', s, flags=re.IGNORECASE)

    # case with acronym preservation
    s_tc = _titlecase_preserving_acronyms(s)

    # build a simple canonical key (lower, spacing unified) for mapping
    key = s_tc.lower()
    key = key.replace('.', '').replace(',', '').strip()
    key = re.sub(r'\s+', ' ', key)
    mapped = CANON.get(key, None)
    if mapped:
        s_tc = mapped

    s_tc = _final_tidy(s_tc)

    # final guard: drop if it still looks wrong
    if not s_tc or len(s_tc) < 2:
        return None

    return s_tc

# --- apply to your df ---
# make a new clean column (safer); drop NAs if you don't want them in viz
books_df_cleaned['Publisher'] = books_df_cleaned['Publisher'].apply(clean_publisher)
books_df_cleaned = books_df_cleaned.dropna(subset=['Publisher'])
books_df_cleaned = books_df_cleaned.reset_index(drop=True)

In [29]:
books_df_cleaned['Publisher'].head(100).unique()

array(['Oxford University Press', 'Harper Flamingo Canada',
       'Harper Perennial', 'Farrar Straus Giroux',
       'W. W. Norton & Company', 'Putnam', 'Berkley Publishing Group',
       'Audioworks', 'Random House', 'Scribner', 'Emblem Editions',
       'Citadel Press', 'House Of Anansi Press', 'Mira Books',
       'Health Communications', 'Brilliance Audio - Trade',
       'Kensington Publishing Corp', 'River City Pub', 'Dell', 'Plume',
       'Three Rivers Press', 'Ryland Peters & Small LTD', 'Cypress House',
       'Harper Entertainment', 'Scholastic', 'Aladdin', 'Ballantine',
       'Random House Trade Paperbacks', 'Goldmann', 'Alfred A. Knopf',
       'Little, Brown And Company', 'HarperCollins', 'Avon', 'Bantam',
       'Fireside', 'Harper Torch', 'Pocket', 'Tor Books',
       'Tyndale House Publishers', 'Harvest Books',
       'Chambers Harrap Publishers LTD', 'Speed Graphics',
       'Sunflower Press', 'Laurel Leaf', 'Gallimard',
       'Prentice Hall (k-12)', 'Ntc/contempor

In [30]:
# common first names to help detect "Last First" swaps
_COMMON_FIRSTS = {
    "john","jane","james","robert","michael","william","david","richard","thomas","charles","joseph",
    "mary","patricia","linda","barbara","elizabeth","jennifer","maria","susan","margaret","sarah",
    "daniel","paul","mark","donald","george","kenneth","steven","edward","brian","ronald","anthony",
    "kevin","jason","matthew","gary","timothy","jose","larry","jeffrey","frank","scott","eric",
    "andrew","stephen","raymond","gregory","joshua","jerry","dennis","walter","patrick","peter",
    "harry","helen","emma","sophie","nicole","anna","karen","carol","ruth","amy","diane","julie",
    "lois","laura","ian","louis","luke","nick","joan","jack","jean","anne","anna","hugh","keith",
    "olivia","tanya","sylvia","rachel","megan","katherine","kathleen","edward","arthur","alan",
    "victoria","julia","susan","alison","allison","judith","joanna","alex","alexander","niccolo",
    "vladimir","paolo","peter","emily","henry","samuel","philip","phillip","andrea"
}

# broadened corporate / series detector
_CORP_PATTERNS = re.compile(
    r'\b('
    r'press|publisher|publishers|publications|company|co\.?|inc\.?|llc|ltd\.?|'   # business
    r'foundation|society|association|committee|institute|dept|department|'        # orgs
    r'library|series|digest|magazine|readers?|family|circle|'                      # series/mags
    r'guides?|workbook|handbook|companion|annual|yearbook'                         # non-person-ish
    r')\b', re.IGNORECASE
)

# things like \" or \'
_ESCAPED_QUOTES = re.compile(r'\\([\'\"])')
# remove bracketed junk: "Name (Editor)" , "Smith [Ed.]" , "Foo {Series}"
_PARENS_BLOCK = re.compile(r'[\(\[\{].*?[\)\]\}]')
# "H+J Rousseau" -> "H. J. Rousseau"
_PLUS_INITIALS = re.compile(r'\b([A-Za-z])\s*\+\s*([A-Za-z])\b')
# split multi-author strings (keep first by default)
_MULTI_AUTHOR_SPLIT = re.compile(r'\s*(?:/|;|&| and |, and )\s+', re.IGNORECASE)

_PLACEHOLDERS = {
    "read","n/a","na","not available","not applicable","varios","various","tk"
}
_EDITORS_OF = re.compile(r'^\s*(the\s+)?editors?\s+of\b', re.IGNORECASE)

# degrees / suffixes to strip
_DEGREES = re.compile(
    r'(?:(?<!\S)(dr|mrs|mr|ms|prof)\.?\s+)|'          # titles at start
    r'(\s*,?\s*(ph\.?\s*d\.?|m\.?\s*d\.?|md|mba|jd|esq\.?|dphil|dd|dds|rn|iii|ii|iv|jr\.?|sr\.?)\s*$)',
    re.IGNORECASE
)

_PARTICLES_LOWER = {"de","del","de la","de los","du","da","dos","van","von","der","den","la","le","di"}

_INITIAL = re.compile(r'^[A-Z]$')
_INITIAL_WITH_DOT = re.compile(r'^[A-Z]\.$')

def _tidy_spaces(s: str) -> str:
    return re.sub(r'\s+', ' ', s).strip()

def _fix_apostrophes(s: str) -> str:
    # normalize weird "O.'dell" -> "O'Dell"
    s = re.sub(r"\bO\.\s*'([a-z])", lambda m: "O'" + m.group(1).upper(), s)
    # normalize smart/odd quotes
    s = s.replace("’", "'").replace("`", "'")
    # Title-case the letter after O'
    s = re.sub(r"\bO'([a-z])", lambda m: "O'" + m.group(1).upper(), s)
    # Fix weird caps like O.'Dell already handled; also d'Este, D'Estate
    s = re.sub(r"\b([A-Za-z])'([a-z])", lambda m: m.group(1) + "'" + m.group(2).upper(), s)
    return s

def _fix_mc_names(token: str) -> str:
    # Join "Mc Murtry" -> "McMurtry"
    token = re.sub(r'\bMc\s+([A-Za-z])', lambda m: 'Mc' + m.group(1).upper(), token)
    # "Mccarthy" -> "McCarthy"
    token = re.sub(r'\bMcc([a-z])', lambda m: 'McC' + m.group(1), token)
    return token

def _ensure_initials(tokens):
    out = []
    for t in tokens:
        core = t.strip(".")
        if len(core) == 1 and core.isalpha():
            out.append(core.upper() + ".")
        else:
            out.append(t)
    return out

def _titlecase_name_piece(piece: str) -> str:
    # keep hyphenated and apostrophe chunks titlecased appropriately
    def tc(word: str) -> str:
        if _INITIAL.match(word) or _INITIAL_WITH_DOT.match(word):
            return word.upper() if len(word) == 1 else word.upper()[0] + "."
        return word[:1].upper() + word[1:].lower()
    # handle apostrophes (O'Dell) and hyphens (Jean-Baptiste)
    piece = "-".join(tc(p) for p in piece.split("-"))
    piece = "'".join(tc(p) for p in piece.split("'"))
    # Mc/Mac tweaks after basic case
    piece = _fix_mc_names(piece)
    return piece

def _smart_titlecase_full(name: str) -> str:
    parts = name.split()
    parts = _ensure_initials(parts)
    # rebuild with particles lowered except first/last when standalone multiword particles
    final = []
    i = 0
    while i < len(parts):
        token = parts[i]
        raw = token
        low = raw.lower().strip(".")
        # multiword particles like "de la"
        if i+1 < len(parts) and f"{low} {parts[i+1].lower().strip('.')}" in _PARTICLES_LOWER:
            combo = f"{low} {parts[i+1].lower().strip('.')}"
            final.append(combo)
            i += 2
            continue
        if low in _PARTICLES_LOWER:
            final.append(low)
        else:
            final.append(_titlecase_name_piece(raw))
        i += 1
    # ensure first and last token not left in lowercase particle by accident
    if final:
        final[0] = _titlecase_name_piece(final[0]) if final[0].lower() in _PARTICLES_LOWER else final[0]
    if len(final) > 1 and final[-1].lower() in _PARTICLES_LOWER:
        final[-1] = _titlecase_name_piece(final[-1])
    return " ".join(final)

def _looks_corporate(s: str) -> bool:
    if _EDITORS_OF.search(s):
        return True
    if _CORP_PATTERNS.search(s):
        return True
    if s.strip().lower() in _PLACEHOLDERS:
        return True
    return False

def _strip_degrees_titles(s: str) -> str:
    prev = None
    while s != prev:
        prev = s
        s = _DEGREES.sub(lambda m: '' if m.group(0) else '', s).strip(" ,")
    return s

def _flip_last_first(s: str) -> str:
    # "Last, First M." -> "First M. Last"
    if "," in s:
        parts = [p.strip() for p in s.split(",")]
        if len(parts) >= 2:
            left = parts[0]
            right = " ".join(parts[1:]).strip()
            if left and right:
                return f"{right} {left}"
    # "Kipling Rudyard" -> "Rudyard Kipling" if second looks like first name
    toks = s.split()
    if len(toks) == 2:
        a, b = toks[0], toks[1]
        if b.lower().strip(".") in _COMMON_FIRSTS and a[0].isupper() and b[0].isupper():
            return f"{b} {a}"
    return s


def clean_author(author: str, *, keep_first_author: bool = True) -> str | None:
    """Return a cleaned, person-like author name or None if it looks non-person/placeholder."""
    if not isinstance(author, str) or not author.strip():
        return None

    s = author.strip()

    # 0) Drop bracketed chunks early (e.g., "(Editor)", "[Ed.]", "{Series}")
    s = _PARENS_BLOCK.sub(" ", s)

    # 1) Unescape things like \" and \'
    s = _ESCAPED_QUOTES.sub(r'\1', s)

    # 2) Normalize HTML & stray entities and Unicode
    s = s.replace("&amp;", "&")
    s = unidecode(s)

    # 3) If multi-author string, optionally keep the first person only
    if keep_first_author and _MULTI_AUTHOR_SPLIT.search(s):
        s = _MULTI_AUTHOR_SPLIT.split(s, maxsplit=1)[0]

    # 4) Quick tidy of spacing
    s = _tidy_spaces(s)

    # 5) Obvious non-persons / placeholders
    if _looks_corporate(s) or s.lower() in _PLACEHOLDERS:
        return None

    # 6) Strip degrees/titles at ends or start
    s = _strip_degrees_titles(s)

    # 7) Flip "Last, First" or common "Last First" cases
    s = _flip_last_first(s)

    # 8) Normalize weird punctuation/casing
    #    - "H+J" -> "H. J."
    s = _PLUS_INITIALS.sub(lambda m: f"{m.group(1).upper()}. {m.group(2).upper()}.", s)
    #    - tidy spaced periods, normalize apostrophes & Mc/Mac
    s = _fix_apostrophes(s)
    s = re.sub(r'\s*\.\s*\.\s*', '..', s)
    s = _fix_mc_names(s)
    s = _tidy_spaces(s)

    # 9) Standardize initials spacing: "J R R Tolkien" -> "J. R. R. Tolkien"
    tokens = _ensure_initials(s.split())
    s = " ".join(tokens)

    # 10) Final smart titlecase with particles handled
    s = _smart_titlecase_full(s).strip(" ,.;-")
    s = _tidy_spaces(s)

    # 11) Prune obvious single-word junk
    bad_singletons = {"read", "christie", "watterson", "barrie", "aawdri", "awdry"}
    if " " not in s and s.lower() in bad_singletons:
        return None

    # 12) Minimum signal (avoid “Abc”, “A”)
    if len(s) < 3:
        return None

    return s or None

In [31]:
books_df_cleaned['Book-Author'] = books_df_cleaned['Book-Author'].apply(clean_author)
books_df_cleaned = books_df_cleaned.dropna(subset=['Book-Author'])
books_df_cleaned = books_df_cleaned.reset_index(drop=True)

In [32]:
books_df_cleaned['Book-Author'].head(100).unique()

array(['Mark P. O. Morford', 'Richard Bruce Wright', "Carlo D'Este",
       'Gina Bari Kolata', 'E. J. W. Barber', 'Amy Tan', 'Robert Cowley',
       'Scott Turow', 'David Cordingly', 'Ann Beattie',
       'David Adams Richards', 'Adam Lebor', 'Sheila Heti',
       'R. J. Kaiser', 'Jack Canfield', 'Loren D. Estleman',
       'Robert Hendrickson', 'Julia Oliver', 'John Grisham',
       'Toni Morrison', 'The Onion', 'Celia Brooks Brown',
       'J. R. Parrish', 'Mary-kate', 'Robynn Clairday', 'Kathleen Duey',
       'Michael Crichton', 'C.s. Lewis', 'Arthur Phillips',
       'Stephan Jaramillo', 'Eleanor Cooney', 'Charlotte Link',
       'Richard North Patterson', 'Mark Salzman', 'Harper Lee',
       'Laura Hillenbrand', 'Barbara Kingsolver', 'Jo Dereske',
       'Jane Austen', 'Dolores Krieger', 'Anne Rivers Siddons',
       'Dean R. Koontz', 'Mary Higgins Clark', 'Dean Koontz',
       'Patricia Cornwell', 'J.d. Robb', 'Maeve Binchy', 'Laura J. Mixon',
       'Tim Lahaye', 'M.d. Bernie 

In [33]:
ACRONYMS = {
    "USA","US","UK","UAE","UN","EU","NATO","GCSE","SAT","ACT","GMAT","GRE",
    "DC","NYC","L.A.","TV","AI","DNA","RNA","U.S.","U.K.","II","III","IV","VI","VII","VIII","IX","X"
}
ROMAN_RE = re.compile(r'^(?=[MDCLXVI])M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$', re.IGNORECASE)

# broader “series/edition” hints (added Shadowrun, Dragonlance, Boxed Set, etc.)
SERIES_HINTS = [
    'series','omnibus','library','edition','ed.','large print','paperback','hardcover',
    'voyager ser','lost classics','collection','collector','vol.','volume','vol','box set','boxed set',
    "world's classics",'classic','guild','annotated','unabridged','illustrated','mass market',
    'shadowrun','dragonlance','forgotten realms','star wars','lotr','lord of the rings'
]

SMALL_WORDS = {
    'a','an','and','as','at','but','by','for','in','nor','of','on','or','per','the','to','via',
    # Spanish small words
    'de','del','la','el','los','las','y','o','en','un','una','unos','unas','por','con','sin','al'
}

# obvious junk we should drop if the whole title equals one of these (case-insensitive)
BAD_TITLE_SET = {'null','n/a','na','untitled','unknown','abc'}

# fast checks for “almost nothing” titles
_ONLY_PUNCT_OR_DIGITS = re.compile(r'^[\W_0-9]+$')

_ESCAPED_QUOTES = re.compile(r'\\([\'\"])')               # \"  -> "
_REPEAT_PUNCT = re.compile(r'([!?.,:;]){2,}')             # "!!!" -> "!"
_EXTRA_SPACES = re.compile(r'\s{2,}')                     # collapse multi spaces

def _smart_token_case(token: str) -> str:
    core = token.strip(".,:;!?\"'()[]{}")
    if not core:
        return token
    if core.upper() in ACRONYMS or ROMAN_RE.match(core):
        return token.replace(core, core.upper())
    # keep possessives nice: O'Brien's -> O'Brien's, CHILDREN'S -> Children's
    token = re.sub(r"\'S\b", "'s", token)
    return token

def _strip_series_parens(text: str) -> str:
    """Remove parenthesized/bracketed chunks that look like series/edition/format metadata."""
    def looks_like_series(chunk: str) -> bool:
        c = chunk.lower().strip()
        if any(h in c for h in SERIES_HINTS):
            return True
        # heuristics: lots of non-words or mostly meta words
        meta = {'novel','paperback','hardcover','edition','volume','vol','book','books',
                'spanish','french','german','italian','translated','translation',
                'complete','illustrated','annotated','revised','abridged'}
        words = re.findall(r"[a-zA-Z]+", c)
        if not words:
            return True
        score = sum(w in meta for w in words)/len(words)
        return score >= 0.5

    s = text
    prev = None
    # iterate until stable (handles nested/adjacent brackets)
    while s != prev:
        prev = s
        s = re.sub(r'\(([^()]*)\)', lambda m: '' if looks_like_series(m.group(1)) else m.group(0), s)
        s = re.sub(r'\[([^\[\]]*)\]', lambda m: '' if looks_like_series(m.group(1)) else m.group(0), s)
        s = re.sub(r'\{([^{}]*)\}',  lambda m: '' if looks_like_series(m.group(1)) else m.group(0), s)
        s = _EXTRA_SPACES.sub(' ', s).strip()
    return s

def _fix_contractions(s: str) -> str:
    # Children'S -> Children's, It'S -> It's
    return re.sub(r"\b([A-Za-z]+)'([A-Z])([a-z]*)\b",
                  lambda m: f"{m.group(1)}'{m.group(2).lower()}{m.group(3)}", s)

def _fix_spanish_articles(tokens):
    out = []
    for i, t in enumerate(tokens):
        core = t.strip(".,:;!?\"'()[]{}")
        if core.upper() == "LA" and i+1 < len(tokens) and tokens[i+1][:1].islower():
            t = t.replace(core, "La")
        out.append(t)
    return out

def _apply_small_words_title_case(tokens):
    if not tokens: 
        return tokens
    new = []
    for i, t in enumerate(tokens):
        core = t.strip(".,:;!?\"'()[]{}")
        low = core.lower()
        if 0 < i < len(tokens)-1 and low in SMALL_WORDS and core.upper() not in ACRONYMS and not ROMAN_RE.match(core):
            t = t.replace(core, low)
        new.append(t)
    return new

def _capitalize_after_punct(s: str) -> str:
    # after colon or em dash, capitalize next letter
    s = re.sub(r'(:\s+)([a-z])', lambda m: m.group(1)+m.group(2).upper(), s)
    s = re.sub(r'(—\s+)([a-z])', lambda m: m.group(1)+m.group(2).upper(), s)
    return s

def _strip_trailing_series_words(s: str) -> str:
    return re.sub(r'\b(?:collection|collections|omnibus|box set|boxed set|annotated edition)\b\.?$', '', s, flags=re.IGNORECASE).strip()

def _spanish_specific_fixes(s: str) -> str:
    s = re.sub(r'\bEl Senor De Los Anillos\b', 'El Senor de los Anillos', s, flags=re.IGNORECASE)
    s = re.sub(r'\bLa Comunidad Del Anillo\b', 'La Comunidad del Anillo', s, flags=re.IGNORECASE)
    s = re.sub(r'\bLas Dos Torres\b', 'Las Dos Torres', s, flags=re.IGNORECASE)
    s = re.sub(r'\bEl Retorno Del Rey\b', 'El Retorno del Rey', s, flags=re.IGNORECASE)
    return s

# --- main cleaner ---

def clean_title(title: str, drop_series: bool = True) -> str | None:
    """Return a normalized, human title or None if it looks like garbage."""
    if not isinstance(title, str) or not title.strip():
        return None

    s = title.strip()

    # obvious mojibake — punt it
    if re.search(r'[ÃÂð¼½¾þÿÞß]', s):
        return None

    # normalize escapes/encoding
    s = _ESCAPED_QUOTES.sub(r'\1', s)     # \" -> "
    s = unidecode(s)
    s = re.sub(r'\s+', ' ', s).strip()

    # ditch total placeholders
    if s.lower() in BAD_TITLE_SET:
        return None

    # punctuation hygiene
    s = _REPEAT_PUNCT.sub(lambda m: m.group(1), s)
    s = re.sub(r'\s*:\s*', ': ', s)
    s = re.sub(r'\s*/\s*', ' / ', s)
    s = re.sub(r'\s+([,.;:!?])', r'\1', s)
    s = re.sub(r'([(\[]) +', r'\1', s)
    s = re.sub(r' +([)\]])', r'\1', s)
    # hyphen spacing: letters around hyphen stick; digit-digit becomes en-dash
    s = re.sub(r'(\d)\s*-\s*(\d)', r'\1–\2', s)
    s = re.sub(r'([A-Za-z])\s*-\s*([A-Za-z])', r'\1-\2', s)

    # contractions
    s = _fix_contractions(s)

    # strip parenthesized/bracketed series/format
    if drop_series:
        s = _strip_series_parens(s)

    # token-level casing
    tokens = s.split(' ')
    tokens = [_smart_token_case(t) for t in tokens]
    tokens = _fix_spanish_articles(tokens)
    tokens = _apply_small_words_title_case(tokens)
    s = ' '.join(tokens)

    # polish
    s = _capitalize_after_punct(s)
    s = _strip_trailing_series_words(s)
    s = _spanish_specific_fixes(s)
    s = _EXTRA_SPACES.sub(' ', s).strip(' -.,;:').strip()

    # final safety: drop if still garbagey
    if not s or _ONLY_PUNCT_OR_DIGITS.match(s):
        return None
    if len(re.sub(r'[^A-Za-z]', '', s)) < 2:      # not enough letters to be a book
        return None

    return s

In [34]:
books_df_cleaned['Book-Title'] = books_df_cleaned['Book-Title'].apply(clean_title)
books_df_cleaned = books_df_cleaned.dropna(subset=['Book-Title'])
books_df_cleaned = books_df_cleaned.reset_index(drop=True)

In [35]:
books_df_cleaned['Book-Title'].head(100).unique()

array(['Classical Mythology', 'Clara Callan', 'Decision in Normandy',
       'Flu: The Story of the Great Influenza Pandemic of 1918 and the Search for the Virus That Caused It',
       'The Mummies of Urumchi', "The Kitchen God's Wife",
       "What If: The World's Foremost Military Historians Imagine What Might Have Been",
       'Pleading Guilty',
       'Under the Black Flag: The Romance and the Reality of Life Among the Pirates',
       "Where You'll Find Me: And Other Stories",
       'Nights Below Station Street',
       "Hitler's Secret Bankers: The Myth of Swiss Neutrality During the Holocaust",
       'The Middle Stories', 'Jane Doe',
       "A Second Chicken Soup for the Woman's Soul", 'The Witchfinder',
       'More Cunning Than Man: A Social History of Rats and Man',
       'Goodbye to the Buttermilk Sky', 'The Testament',
       'Beloved (Plume Contemporary Fiction)',
       "Our Dumb Century: The Onion Presents 100 Years of Headlines From America's Finest News Source",
 

In [36]:
# Drop rows with short or clearly invalid city names
invalid_city_patterns = ['-', '.', '...', 'X', '_____', '~*~', '---------']

users_df = users_df[~users_df['City'].isin(invalid_city_patterns)]

users_df = users_df[users_df['City'].str.len() > 2]

users_df['City'] = users_df['City'].str.replace(r'\s*\(.*?\)', '', regex=True).str.strip().str.title()

users_df['City'] = users_df['City'].str.split('/').str[0].str.strip()

# Define function to detect invalid city entries
def is_invalid_city(city):
    if pd.isna(city):
        return True
    city = str(city).strip()
    return (
        city.lower() in garbage_set or
        bool(re.match(r"^\d{4,6}$", city)) or                 # exact postal code
        bool(re.match(r"^\d+\s", city)) or                    # starts with a number
        bool(re.search(r"\d{4,6}", city)) or                  # contains long number
        bool(re.match(r"^\d{5}\s+\w+", city)) or              # "23564 Lubeck"
        len(city) < 3                                         # too short
    )

# Known garbage city values (lowercase for easier comparison)
garbage_values = {
    'unknown', 'unk', 'n/a', 'na', 'none', 'null',
    'abc', 'a town', '', '.', '..', '00000', '0'
}

# Convert to lowercase for consistent matching
garbage_set = set(val.lower() for val in garbage_values)

# Apply the cleaning: set invalid cities to <NA>
users_df['City'] = users_df['City'].apply(
    lambda x: pd.NA if is_invalid_city(x) else str(x).strip().title()
)

# Optional final cleanup (just in case)
users_df['City'] = users_df['City'].str.replace(r"[^a-zA-ZÀ-ÿ\s\'\-]", "", regex=True).str.strip()

users_df[['City', 'State', 'Country']] = users_df[['City', 'State', 'Country']].replace(
    to_replace=r'^\s*$|^[Uu]nknown$|^,\s*$',
    value=pd.NA,
    regex=True
)

users_df = users_df.dropna(subset=['City', 'State', 'Country'])
users_df = users_df.reset_index(drop=True)
users_df.head()

,User-ID,Age,City,State,Country
0,2,18,Stockton,california,usa
1,10,26,Albacete,wisconsin,spain
2,20,19,Langhorne,pennsylvania,usa
3,36,24,Montreal,quebec,canada
4,38,34,Viterbo,lazio,italy


In [37]:
users_df['city_corrupt'] = users_df['City'].apply(has_non_ascii)
users_df['state_corrupt'] = users_df['State'].apply(has_non_ascii)
users_df['country_corrupt'] = users_df['Country'].apply(has_non_ascii)

users_df['short_city'] = users_df['City'].str.strip().str.len() <= 2
users_df['short_state'] = users_df['State'].str.strip().str.len() <= 2
users_df['short_country'] = users_df['Country'].str.strip().str.len() <= 2

In [38]:
# Define flag columns and count the number of them to drop fields if there are more than or equal to 2 flags
flags = ['city_corrupt', 'state_corrupt', 'country_corrupt', 'short_city', 'short_state', 'short_country']
users_df['bad_count'] = users_df[flags].sum(axis=1)
users_df_cleaned = users_df[users_df['bad_count'] < 2].drop(columns=flags + ['bad_count']).reset_index(drop=True)

In [39]:
print("Before:", len(users_df))
print("After:", len(users_df_cleaned))
print("Dropped:", len(users_df) - len(users_df_cleaned))

Before: 59898
After: 59555
Dropped: 343


In [40]:
users_df_cleaned['City'] = users_df_cleaned['City'].str.title().str.strip()
users_df_cleaned['State'] = users_df_cleaned['State'].str.title().str.strip()
users_df_cleaned['Country'] = users_df_cleaned['Country'].str.title().str.strip()

In [41]:
users_df_cleaned.head()

,User-ID,Age,City,State,Country
0,2,18,Stockton,California,Usa
1,10,26,Albacete,Wisconsin,Spain
2,20,19,Langhorne,Pennsylvania,Usa
3,36,24,Montreal,Quebec,Canada
4,38,34,Viterbo,Lazio,Italy


In [42]:
# Keep only the last part after the last comma (assuming it's the country)
users_df_cleaned['Country'] = users_df_cleaned['Country'].str.split(',').str[-1].str.strip()

# Remove extra quotes and non-letter characters
users_df_cleaned['Country'] = users_df_cleaned['Country'].str.replace(r'[^a-zA-Z\s]', '', regex=True)

country_fixes = {
    'Everywhere And Anywhere': pd.NA,
    'Here And There': pd.NA,
    'Hungary And Usa': pd.NA,
    'The': pd.NA,
    'United State': 'United States',
    'Us': 'United States',
    'USA': 'United States',
    'Usa': 'United States',
    'USA"': 'United States',
    'Usa & Canada': 'United States',
    'Phillipines': 'Philippines',
    'Philippines"': 'Philippines',
    'Uae': 'United Arab Emirates',
    'U.A.E': 'United Arab Emirates',
    'UAE': 'United Arab Emirates',
    'UK': 'United Kingdom',
    'Uk': 'United Kingdom',
    'European Union': pd.NA,
    'Eu': pd.NA,
    'England': 'United Kingdom',
    'Great Britain': 'United Kingdom',
    'España': 'Spain',
    'España"': 'Spain',
    'Spain"': 'Spain',
    'France"': 'France',
    'Portugal"': 'Portugal',
    'Switzerland"': 'Switzerland',
    'New Zealand"': 'New Zealand',
    'Netherlands"': 'Netherlands',
    'Norway"': 'Norway',
    'America': 'United States',
    'Ysa': 'United States',
    'Universe': pd.NA,
    '"N/A"': pd.NA,
    "NA": pd.NA,
    '\\"N/A\\""': pd.NA,
    'Alachua': pd.NA,
    'Polk': pd.NA,
    'Quit': pd.NA,
    'Pender': pd.NA,
    'Urugua': 'Uruguay',
    'Cherokee': pd.NA,
    'Galiza': pd.NA,
    'Catalunya': 'Spain',
    'Catalunya Spain': 'Spain',
    'Deutschland': 'Germany',
    'Euskal Herria': pd.NA,
    'Catalonia': 'Spain',
    'Catalonia Spain': 'Spain',
    'Italia': 'Italy',
    'Canary Islands, Spain': 'Spain',
    'Turkey"': 'Turkey',
    'EspaA': 'Spain',
    'Basque Country': 'Spain',
    'Lleida': 'Spain',
    'Orense': 'Spain',
    'Madrid': 'Spain',
    'Vicenza': 'Italy',
    'Lombardia': 'Italy',
    'La Belgique': 'Belgium',
    'La France': 'France',
    'La Suisse': 'Switzerland',
    'Brasil': 'Brazil',
    'PRChina': 'China',
    'GuineaBissau': 'Guinea-Bissau',
    'The Netherlands': 'Netherlands',
    'Holland': 'Netherlands',
    'Tobago': 'Trinidad and Tobago',
    'Guernsey': 'United Kingdom',
    'Jersey': 'United Kingdom',
    'Alderney': 'United Kingdom',
    'Wales': 'United Kingdom',
    'Scotland': 'United Kingdom',
    'Ouranos': pd.NA,
    'Far Away': pd.NA,
    'Galiza Neghra': pd.NA,
    'Burlington': pd.NA,
    'Hernando': pd.NA,
    '': pd.NA,
    'US Virgin Islands': 'United States',
    'Bermuda': 'United Kingdom',
    'Cayman Islands': 'United Kingdom',
    'Guam': 'United States',
    'Macau': 'China',
    'Puerto Rico': 'United States',
    'Yugoslavia': pd.NA, 
    'Antarctica': pd.NA,
    'Usa  Canada': pd.NA,
    'Trinidad and Tobago': 'Trinidad And Tobago'  # unify casing if needed
}


# Apply fixes
users_df_cleaned['Country'] = users_df_cleaned['Country'].replace(country_fixes)
users_df_cleaned = users_df_cleaned.dropna(subset=['Country'])
users_df_cleaned = users_df_cleaned.reset_index(drop=True)
users_df_cleaned.head()

,User-ID,Age,City,State,Country
0,2,18,Stockton,California,United States
1,10,26,Albacete,Wisconsin,Spain
2,20,19,Langhorne,Pennsylvania,United States
3,36,24,Montreal,Quebec,Canada
4,38,34,Viterbo,Lazio,Italy


In [43]:
def is_bad_encoding(city):
    if not isinstance(city, str):
        return False
    return bool(re.search(r'[ÃãÂ±ðÃ©Ã¨ÃªÃ¼Ã¶Ã¤ÃŸÃªÃ¢Ã«ÃŽÃ®Ã»ÃƒÃ€Ã¦]', city))

users_df_cleaned['City'] = users_df_cleaned['City'].apply(
    lambda x: None if is_bad_encoding(x) else x
)


# Убираем акценты, диакритику и переводим всё к ASCII
users_df_cleaned['City'] = users_df_cleaned['City'].apply(
    lambda x: unidecode(x) if isinstance(x, str) else x
)

city_fixes = {
    'Slfjk': pd.NA,
    'Mei Foo': pd.NA,
    'Cmch': pd.NA,
    'Moeville': pd.NA,
    'Tuilla De Langreo': pd.NA,
    'Island Lake Il': pd.NA,
    'Ny': 'New York City',
    'Nyc': 'New York City',
    'Sf': 'San Francisco',
    'La': 'Los Angeles',
    'Ft Sill': 'Fort Sill',
    'Ftstewart': 'Fort Stewart',
    'Wien': 'Vienna',
    'Roma': 'Rome',
    'Munchen': 'Munich',
    'Muenchen': 'Munich',
    'Roma': 'Rome',
    'Milano': 'Milan',
    'Londres': 'London',
    'Lisboa': 'Lisbon',
    'Koeln': 'Cologne',
    'Wien': 'Vienna',
    'Frankfurt Am Main': 'Frankfurt',
    'Stuttgart - West': 'Stuttgart',
    'Ccp': pd.NA,
    'Koln': 'Cologne',
    'Sao Paolo': 'Sao Paulo',
    'Distrito Federal': 'Mexico City',
    'Albq': 'Albuquerque',
    'Cambridge, UK': 'Cambridge',
    'Saint Louis': 'St. Louis',
    'Saint Paul': 'St. Paul',
    'Newcastle Upon Tyne': 'Newcastle',
    'Winston Salem': 'Winston-Salem',
    'New York': 'New York City',
    'Washington Dc': 'Washington',
    'Newcastle Upon  Tyne': 'Newcastle Upon Tyne'
}

# Стандартизируем: убираем пробелы, приводим к Title Case
users_df_cleaned['City'] = (
    users_df_cleaned['City']
    .astype(str)
    .str.strip()
    .str.title()
)

users_df_cleaned['City'] = users_df_cleaned['City'].str.split(',').str[0]
users_df_cleaned['City'] = users_df_cleaned['City'].replace(city_fixes)
users_df_cleaned = users_df_cleaned.dropna(subset=['City'])
users_df_cleaned = users_df_cleaned[users_df_cleaned['City'].notna()]

In [44]:
users_df_cleaned.head()

,User-ID,Age,City,State,Country
0,2,18,Stockton,California,United States
1,10,26,Albacete,Wisconsin,Spain
2,20,19,Langhorne,Pennsylvania,United States
3,36,24,Montreal,Quebec,Canada
4,38,34,Viterbo,Lazio,Italy


In [45]:
ratings_df.rename(columns={'User-ID': 'User_ID', 'Book-Rating': 'Book_Rating'}, inplace=True)
users_df_cleaned.rename(columns={'User-ID': 'User_ID'}, inplace=True)
books_df_cleaned.rename(columns={'Book-Title': 'Book_Title', 'Book-Author': 'Book_Author', 'Year-Of-Publication': 'Year_Of_Publication'}, inplace=True)

In [46]:
books_df_cleaned.to_csv("books_cleaned_2.csv", index=False)
ratings_df.to_csv("ratings_cleaned_2.csv", index=False)
users_df_cleaned.to_csv("users_cleaned_2.csv", index=False)